In [1]:
%pip install torch transformers
%pip install fugashi ipadic

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from transformers import BertForMaskedLM, BertJapaneseTokenizer
import torch
import csv
import json

c:\WorkSpace\Practice\env2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
#model_name = "cl-tohoku/bert-base-japanese"
#model_name = "cl-tohoku/bert-base-japanese-v3"
#model_name = "cl-tohoku/bert-base-japanese-char-v3"
#model_name = "cl-tohoku/bert-large-japanese-v2"
model_name = "cl-tohoku/bert-large-japanese-char-v2"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

c:\WorkSpace\Practice\env2\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\thisi\.cache\huggingface\hub\models--cl-tohoku--bert-large-japanese-char-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at cl-tohoku/bert-large-japanese-char-v2 were not used when ini

In [15]:
def tokenize_with_mask(text, tokenizer):
    tokens = tokenizer.tokenize(text)
    masked_tokens = []
    for i in range(len(tokens)):
        if tokens[i] not in ['[CLS]', '[SEP]']:
            masked_tokens.append(tokens[:i] + ['[MASK]'] + tokens[i+1:])
    return masked_tokens

def calculate_score(masked_tokens, tokenizer, model):
    scores = []
    for tokens in masked_tokens:
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        tensor_input = torch.tensor([input_ids])
        with torch.no_grad():
            outputs = model(tensor_input, labels=tensor_input)
            loss = outputs[0]
            scores.append(loss.item())
    return sum(scores) / len(scores)

In [16]:
#テキストの読み込み
with open('C:/Users/thisi/OneDrive/デスクトップ/新規 テキスト ドキュメント.txt', 'r', encoding='utf-8') as file:
    texts = file.readlines()

results = []
for text in texts[:100]: 
    text = text.strip()
    masked_tokens = tokenize_with_mask(text, tokenizer)
    score = calculate_score(masked_tokens, tokenizer, model)
    results.append({'text': text, 'score': score})

In [17]:
# CSVファイルに保存
with open('results5.csv', 'w', encoding='utf-8', newline='') as csv_file:
    fieldnames = ['text', 'score']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for result in results:
        writer.writerow(result)